# Google Colab 에 REST API Server 구축하기

Google Colab 을 REST API 를 위한 Back-end server 처럼 구동합니다.
이를 위해서 ngrok 을 사용 하는데, 특성상 8시간만 링크를 유지할 수 있기 때문에 DEMO 용도로 사용하는게 좋습니다.

---

## 필요 파일 설치

서버를 구동하기 위해 필요한 기본 패키지를 Colab에 설치합니다.

In [1]:
!pip install Flask
!pip install flask-ngrok
!pip install flask-restx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 24.1 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 


## 패키지 임포트

In [ ]:
import os
import random
import base64
from io import BytesIO
from threading import Thread

import numpy

from flask import Flask, jsonify
from flask_restx import Api, Resource
from flask_ngrok import run_with_ngrok
from tqdm import tqdm
from PIL import Image

## Flask App 생성

Flask App을 생성하고 초기화합니다.

*참고로 run_with_ngrok 은 ngrok 으로 외부에서 접근하게 하기위한 목적으로 필요합니다.*

In [ ]:
app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)

## Google 계정 연동
Google 계정을 연동해서 Google-Drive 를 연결합니다.

Google-Drive 는 마치 서버처럼 사용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Image 포장하기
몇 개의 Random 한 Image 들을 불러와서 REST API 를 사용해서 보내는 함수입니다.
참고로 Google Drive 내에 이미지를 저장하는 서버처럼 사용할 위치는 아래 ROOT_DIR에 입력하면 된다.

In [ ]:
ROOT_DIR = '/content/drive/MyDrive/MetaArtServer'
for root, _, files in os.walk(ROOT_DIR):
  for f in files:
    print(os.path.join(root, f))

In [ ]:
IMG_FORMAT = ['.jpg', '.bmp', '.png']

def get_images(count:int, path_=''):
    try:
        # 디렉토리 내에서 count 만큼의 이미지를 랜덤하게 가져옴
        img_path = os.path.join(ROOT_DIR, path_)
        imgs = []
        for root, _, files in os.walk(img_path):
          imgs += [os.path.join(root, f) for f in files \
                   if os.path.splitext(f)[-1] in IMG_FORMAT]
        if count > len(imgs):
          count = len(imgs)
        sample_imgs = random.sample(imgs, count)
        print(f'Sampling the {count} images')
        # 응답 저장
        response = []
        for i, im_path in tqdm(enumerate(sample_imgs), desc='wrapping images'):
            img = Image.open(im_path)
            buffered = BytesIO()
            img.save(buffered, format="JPEG")
            img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
            response.append(img_str)
        # Logging
        for i, img_str in enumerate(response):
            print(f'send({i}) => {img_str}')
        return response
    except Exception as e:
        print('Error occur in get images!\n', e)
        return {'Error': e}, 500

## Flask 블루 프린트 구성
Image 를 Flask와 REST로 보낼 수 있는 router를 구성합니다.

In [ ]:
@api.route('/images/<int:num>')
class images(Resource):
    def get(self, num):
        return get_images(count=num)

@api.route('/healthz')
class health(Resource):
    def get(self):
        return 'return_health', 200

## Server 실행
Flask 명령으로 서버를 실행합니다.

In [ ]:
server = Thread(target=app.run)
server.start()

------
# Client 구성
Colab 내 Test 를 위해 Client 를 구성합니다. Client 로 먼저 host 주소에 health 체크 명령을 보냅니다.

In [ ]:
import requests

HOST_SERVER = 'http://127.0.0.1:5000'
res = requests.get(HOST_SERVER + '/healthz')
print(res.content)

## Client -> Server 요청

REST API를 활용해 Image를 가져옵니다.

In [ ]:
res = requests.get(HOST_SERVER + '/images/10')

In [ ]:
print(res.content)

## 디코딩
Base64 로 인코딩된 결과값을 디코딩해서 출력해봅니다.

In [ ]:
img_encodes = res.json()
print(img_encodes)